<a href="https://colab.research.google.com/github/jubiology/misc/blob/main/eQTL_loci_allele_counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get allele counts at eQTL loci from all individuals in Burkina Faso



[link text](https://)

In [1]:
%pip install -q malariagen_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.5/302.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import pandas as pd
import numpy as np
import malariagen_data
import allel
import os

In [3]:
ag3 = malariagen_data.Ag3()
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10
Results cache           : None
Cohorts analysis        : 20240418
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 11.0.0
Client location         : Oregon, United States (Google Cloud us-west1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v11.0.0/Ag3.html

In [4]:
try:
    from google.colab import drive
    drive.mount("drive")
except ImportError:
    pass

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [9]:
input_dir = "drive/MyDrive/malariagen_allele_counts"
results_dir = "drive/MyDrive/malariagen_allele_counts"

# Working approach
Analysis needs to be split between chromosomes

In [6]:
# set sample sets
sample_sets=["AG1000G-BF-A", "AG1000G-BF-B", "AG1000G-BF-C"]


Chromosome X

In [10]:
region="X"

# Access SNP positions and alleles data.
ds_snp = ag3.snp_variants(region=region)

# Genomic positions of the SNPs.
pos = ds_snp["variant_position"].values

# Alleles of the SNPs, first column is always the reference allele, other columns are non-reference alleles.
alleles = ds_snp["variant_allele"].values

# Allele counts, columns correspond to alleles shown above.
ac = ag3.snp_allele_counts(region=region, sample_sets=sample_sets)

# Define the input and output file paths
input_file_path = os.path.join(input_dir, 'eQTLs_in_goi_locX.txt')
output_file_path = os.path.join(results_dir, 'allelecount_results_ChrX.tsv')

# Read regions from the input file
with open(input_file_path, 'r') as file:
    positions_of_interest = [line.strip().strip('"') for line in file]

idx = allel.SortedIndex(pos)

# Locate the positions of SNPs you are interested in.
loc = idx.locate_keys(positions_of_interest)

# Filter the data to select the SNPs of interest.
alleles_loc = alleles[loc]
pos_loc = pos[loc]
ac_loc = ac[loc]

# Concatenate the arrays into a single structured array
combined_data = np.column_stack((pos_loc, alleles_loc, ac_loc))

# Define the header
header = ['chr', 'position'] + [f'allele{i}' for i in range(alleles_loc.shape[1])] + [f'count{i}' for i in range(ac_loc.shape[1])]

# Convert the NumPy array to a DataFrame
df = pd.DataFrame(combined_data, columns=header[1:])

# Clean up the DataFrame by removing b' and trailing '
df = df.applymap(lambda x: x.decode('utf-8').strip("b'") if isinstance(x, bytes) else x)

# Add the contig name as the first column
df.insert(0, 'chr', region)

# Save the combined data to a single file
df.to_csv(output_file_path, sep='\t', index=False)

print(f"Results saved to {output_file_path}")

Compute SNP allele counts:   0%|          | 0/1482 [00:00<?, ?it/s]

Results saved to drive/MyDrive/malariagen_allele_counts/allelecount_results_ChrX.tsv


Chromosome 2L

In [11]:
region="2L"

# Access SNP positions and alleles data.
ds_snp = ag3.snp_variants(region=region)

# Genomic positions of the SNPs.
pos = ds_snp["variant_position"].values

# Alleles of the SNPs, first column is always the reference allele, other columns are non-reference alleles.
alleles = ds_snp["variant_allele"].values

# Allele counts, columns correspond to alleles shown above.
ac = ag3.snp_allele_counts(region=region, sample_sets=sample_sets)

# Define the input and output file paths
input_file_path = os.path.join(input_dir, 'eQTLs_in_goi_loc2L.txt')
output_file_path = os.path.join(results_dir, 'allelecount_results_Chr2L.tsv')

# Read regions from the input file
with open(input_file_path, 'r') as file:
    positions_of_interest = [line.strip().strip('"') for line in file]

idx = allel.SortedIndex(pos)

# Locate the positions of SNPs you are interested in.
loc = idx.locate_keys(positions_of_interest)

# Filter the data to select the SNPs of interest.
alleles_loc = alleles[loc]
pos_loc = pos[loc]
ac_loc = ac[loc]

# Concatenate the arrays into a single structured array
combined_data = np.column_stack((pos_loc, alleles_loc, ac_loc))

# Define the header
header = ['chr', 'position'] + [f'allele{i}' for i in range(alleles_loc.shape[1])] + [f'count{i}' for i in range(ac_loc.shape[1])]

# Convert the NumPy array to a DataFrame
df = pd.DataFrame(combined_data, columns=header[1:])

# Clean up the DataFrame by removing b' and trailing '
df = df.applymap(lambda x: x.decode('utf-8').strip("b'") if isinstance(x, bytes) else x)

# Add the contig name as the first column
df.insert(0, 'chr', region)

# Save the combined data to a single file
df.to_csv(output_file_path, sep='\t', index=False)

print(f"Results saved to {output_file_path}")

Compute SNP allele counts:   0%|          | 0/3078 [00:00<?, ?it/s]

Results saved to drive/MyDrive/malariagen_allele_counts/allelecount_results_Chr2L.tsv


Chromosome 2R

In [12]:
region="2R"

# Access SNP positions and alleles data.
ds_snp = ag3.snp_variants(region=region)

# Genomic positions of the SNPs.
pos = ds_snp["variant_position"].values

# Alleles of the SNPs, first column is always the reference allele, other columns are non-reference alleles.
alleles = ds_snp["variant_allele"].values

# Allele counts, columns correspond to alleles shown above.
ac = ag3.snp_allele_counts(region=region, sample_sets=sample_sets)

# Define the input and output file paths
input_file_path = os.path.join(input_dir, 'eQTLs_in_goi_loc2R.txt')
output_file_path = os.path.join(results_dir, 'allelecount_results_Chr2R.tsv')

# Read regions from the input file
with open(input_file_path, 'r') as file:
    positions_of_interest = [line.strip().strip('"') for line in file]

idx = allel.SortedIndex(pos)

# Locate the positions of SNPs you are interested in.
loc = idx.locate_keys(positions_of_interest)

# Filter the data to select the SNPs of interest.
alleles_loc = alleles[loc]
pos_loc = pos[loc]
ac_loc = ac[loc]

# Concatenate the arrays into a single structured array
combined_data = np.column_stack((pos_loc, alleles_loc, ac_loc))

# Define the header
header = ['chr', 'position'] + [f'allele{i}' for i in range(alleles_loc.shape[1])] + [f'count{i}' for i in range(ac_loc.shape[1])]

# Convert the NumPy array to a DataFrame
df = pd.DataFrame(combined_data, columns=header[1:])

# Clean up the DataFrame by removing b' and trailing '
df = df.applymap(lambda x: x.decode('utf-8').strip("b'") if isinstance(x, bytes) else x)

# Add the contig name as the first column
df.insert(0, 'chr', region)

# Save the combined data to a single file
df.to_csv(output_file_path, sep='\t', index=False)

print(f"Results saved to {output_file_path}")

Compute SNP allele counts:   0%|          | 0/3819 [00:00<?, ?it/s]

Results saved to drive/MyDrive/malariagen_allele_counts/allelecount_results_Chr2R.tsv


Chromosome 3L

In [13]:
region="3L"

# Access SNP positions and alleles data.
ds_snp = ag3.snp_variants(region=region)

# Genomic positions of the SNPs.
pos = ds_snp["variant_position"].values

# Alleles of the SNPs, first column is always the reference allele, other columns are non-reference alleles.
alleles = ds_snp["variant_allele"].values

# Allele counts, columns correspond to alleles shown above.
ac = ag3.snp_allele_counts(region=region, sample_sets=sample_sets)

# Define the input and output file paths
input_file_path = os.path.join(input_dir, 'eQTLs_in_goi_loc3L.txt')
output_file_path = os.path.join(results_dir, 'allelecount_results_Chr3L.tsv')

# Read regions from the input file
with open(input_file_path, 'r') as file:
    positions_of_interest = [line.strip().strip('"') for line in file]

idx = allel.SortedIndex(pos)

# Locate the positions of SNPs you are interested in.
loc = idx.locate_keys(positions_of_interest)

# Filter the data to select the SNPs of interest.
alleles_loc = alleles[loc]
pos_loc = pos[loc]
ac_loc = ac[loc]

# Concatenate the arrays into a single structured array
combined_data = np.column_stack((pos_loc, alleles_loc, ac_loc))

# Define the header
header = ['chr', 'position'] + [f'allele{i}' for i in range(alleles_loc.shape[1])] + [f'count{i}' for i in range(ac_loc.shape[1])]

# Convert the NumPy array to a DataFrame
df = pd.DataFrame(combined_data, columns=header[1:])

# Clean up the DataFrame by removing b' and trailing '
df = df.applymap(lambda x: x.decode('utf-8').strip("b'") if isinstance(x, bytes) else x)

# Add the contig name as the first column
df.insert(0, 'chr', region)

# Save the combined data to a single file
df.to_csv(output_file_path, sep='\t', index=False)

print(f"Results saved to {output_file_path}")

Compute SNP allele counts:   0%|          | 0/2584 [00:00<?, ?it/s]

Results saved to drive/MyDrive/malariagen_allele_counts/allelecount_results_Chr3L.tsv


Chromosome 3R

In [14]:
region="3R"

# Access SNP positions and alleles data.
ds_snp = ag3.snp_variants(region=region)

# Genomic positions of the SNPs.
pos = ds_snp["variant_position"].values

# Alleles of the SNPs, first column is always the reference allele, other columns are non-reference alleles.
alleles = ds_snp["variant_allele"].values

# Allele counts, columns correspond to alleles shown above.
ac = ag3.snp_allele_counts(region=region, sample_sets=sample_sets)

# Define the input and output file paths
input_file_path = os.path.join(input_dir, 'eQTLs_in_goi_loc3R.txt')
output_file_path = os.path.join(results_dir, 'allelecount_results_Chr3R.tsv')

# Read regions from the input file
with open(input_file_path, 'r') as file:
    positions_of_interest = [line.strip().strip('"') for line in file]

idx = allel.SortedIndex(pos)

# Locate the positions of SNPs you are interested in.
loc = idx.locate_keys(positions_of_interest)

# Filter the data to select the SNPs of interest.
alleles_loc = alleles[loc]
pos_loc = pos[loc]
ac_loc = ac[loc]

# Concatenate the arrays into a single structured array
combined_data = np.column_stack((pos_loc, alleles_loc, ac_loc))

# Define the header
header = ['chr', 'position'] + [f'allele{i}' for i in range(alleles_loc.shape[1])] + [f'count{i}' for i in range(ac_loc.shape[1])]

# Convert the NumPy array to a DataFrame
df = pd.DataFrame(combined_data, columns=header[1:])

# Clean up the DataFrame by removing b' and trailing '
df = df.applymap(lambda x: x.decode('utf-8').strip("b'") if isinstance(x, bytes) else x)

# Add the contig name as the first column
df.insert(0, 'chr', region)

# Save the combined data to a single file
df.to_csv(output_file_path, sep='\t', index=False)

print(f"Results saved to {output_file_path}")

Compute SNP allele counts:   0%|          | 0/3325 [00:00<?, ?it/s]

Results saved to drive/MyDrive/malariagen_allele_counts/allelecount_results_Chr3R.tsv
